<h1>Query Similarity Search with Embeddings</h1>

<p>This example demonstrates how to use embeddings to calculate the cosine similarity between queries and a sample dataset. We will use a small dataset containing categories like <strong>Smartphones</strong>, <strong>Audio Equipment</strong>, <strong>Kitchen Appliances</strong>, <strong>Athletic Footwear</strong>, and <strong>Home Cleaning</strong>. For each query, we embed the text, calculate the cosine similarity, and retrieve the most similar item.</p>

<h2>Dataset</h2>
<ul>
    <li><strong>Smartphones</strong></li>
    <li><strong>Audio Equipment</strong></li>
    <li><strong>Kitchen Appliances</strong></li>
    <li><strong>Athletic Footwear</strong></li>
    <li><strong>Home Cleaning</strong></li>
</ul>

<h3>Example Process</h3>
<p>For each category, we embedded the text descriptions, calculated cosine similarity, and identified the most similar item based on a query. Below are the queries and their corresponding most similar items:</p>


<h2>Importing Required Packages</h2>

<p>Before starting, ensure you have all the necessary packages installed. If a package is missing, you can install it using <code>pip</code>. Below is the list of required imports for this project:</p>

In [11]:
from sentence_transformers import SentenceTransformer

import torch 
import torch.nn.functional as F
from torch import Tensor

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score, confusion_matrix

from datasets import load_from_disk, Dataset

import numpy as np
import pandas as pd

<h2>Load Dataset and Move Model to GPU (if available)</h2>

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('Metric-AI/armenian-text-embeddings-1',device=device)
dataset = load_from_disk('product_demo_data')

No sentence-transformers model found with name Metric-AI/armenian-text-embeddings-1. Creating a new one with mean pooling.


In [13]:
print(dataset)

Dataset({
    features: ['name', 'description', 'item_section'],
    num_rows: 15
})


<h1>Passage Preprocessing Steps</h1>

<p>In this example, product descriptions are treated as "passages." Each passage is prefixed with <code>passage:</code>. The following preprocessing steps are applied:</p>

In [14]:
dataset = dataset.map(lambda x: {'passage': 'passage: '+x['description']})
dataset = dataset.map(lambda x: {'embedding': model.encode(x['passage'], normalize_embeddings=True)}, batch_size=32)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [15]:
df = dataset.to_pandas()

In [16]:
for i in range(df.shape[0]):
    print(df.iloc[i]['description'])

Apple-ի ֆլագման սմարթֆոն՝ առաջադեմ տեսախցիկի համակարգով, A17 Pro չիպով և տիտանից կորպուսով, որը ունի նորարարական հաշվարկային լուսանկարչական հնարավորություններ:
Բարձր տեսակի Android սմարթֆոն՝ 200MP տեսախցիկով, S Pen ինտեգրացիայով, հզոր Snapdragon պրոցեսորով և լայն էկրանով՝ խաղերի և լուսանկարչության համար:
Պրեմիում Android սմարթֆոն՝ հզոր տեսախցիկով, արհեստական բանականությամբ համալրված հատկություններով և հզոր պրոցեսորով՝ ստեղծագործական փորձառության համար:
Պրեմիում աղմուկը մեկուսացնող անլար ականջակալներ՝ բացառիկ ձայնի որակով, խելացի աղմուկի մեկուսացմամբ և հարմարավետ over-ear դիզայնով՝ աուդիոֆիլների համար:
Անլար աղմուկը մեկուսացնող ականջակալներ՝ ադապտիվ ձայնի կառավարմամբ, հարմարավետ տեղադրմամբ և բացառիկ ձայնի որակով՝ իմերսիվ լսողական փորձառության համար:
Առաջադեմ անլար ականջներսի ականջակալներ՝ ակտիվ աղմուկի մեկուսացմամբ, տարածական ձայնով և Apple էկոհամակարգում անխափան ինտեգրացիայով՝ պրեմիում լսողական փորձառության համար:
Իկոնիկ խոհանոցի սարք՝ հզոր մոտոր, բազմաֆունկցիոնալ կցորդների հնարավորութ

<h1>Query Similarity Calculation</h1>

<p>The following steps are taken in the code to calculate the similarity between queries and product descriptions:</p>

<ol>
    <li><strong>Query Processing:</strong> Each query is prefixed with <code>query:</code> to differentiate it from the product descriptions.</li>
    <li><strong>Embedding Queries:</strong> The processed queries are embedded using a pre-trained model, and the embeddings are normalized.</li>
    <li><strong>Similarity Calculation:</strong> For each query, the cosine similarity is calculated between the query embedding and the embeddings of the products in the dataset.</li>
    <li><strong>Finding the Most Similar Product:</strong> The product with the highest similarity score is selected as the most similar item to the query. The name and description of the most similar product are printed for each query.</li>
</ol>

In [18]:
queries = [
    "բարձրորակ սմարթֆոն՝ հզոր պրոցեսորով",
    "աղմուկը մեկուսացնող անլար ականջակալներ",
    "բազմաֆունկցիոնալ խոհանոցային գործիք",
    "վազքի կոշիկներ",
    "ռոբոտ փոշեկուլ"
]
processed_queries = ['query: ' + query for query in queries]
query_embedding = model.encode(processed_queries, normalize_embeddings=True)

for i, query in enumerate(queries):
    dataset = dataset.map(lambda x: {'similarity_scores': x['embedding']@query_embedding[i]})
    max_row = max(dataset, key=lambda x: x['similarity_scores'])
    
    print(f"Query: {query}")
    print(f"Most similar item in the dataset:")
    print(f"Name: {max_row['name']}")
    print(f"Description: {max_row['description']}")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Query: բարձրորակ սմարթֆոն՝ հզոր պրոցեսորով
Most similar item in the dataset:
Name: Google Pixel 8 Pro
Description: Պրեմիում Android սմարթֆոն՝ հզոր տեսախցիկով, արհեստական բանականությամբ համալրված հատկություններով և հզոր պրոցեսորով՝ ստեղծագործական փորձառության համար:


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Query: աղմուկը մեկուսացնող անլար ականջակալներ
Most similar item in the dataset:
Name: Bose QuietComfort Earbuds II
Description: Անլար աղմուկը մեկուսացնող ականջակալներ՝ ադապտիվ ձայնի կառավարմամբ, հարմարավետ տեղադրմամբ և բացառիկ ձայնի որակով՝ իմերսիվ լսողական փորձառության համար:


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Query: բազմաֆունկցիոնալ խոհանոցային գործիք
Most similar item in the dataset:
Name: KitchenAid Artisan սեղանի խառնիչ
Description: Իկոնիկ խոհանոցի սարք՝ հզոր մոտոր, բազմաֆունկցիոնալ կցորդների հնարավորություն և կլասիկ դիզայն՝ տարբեր գույներով՝ տնային հացաթխիչների և խոհարարական էնտուզիաստների համար:


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Query: վազքի կոշիկներ
Most similar item in the dataset:
Name: Nike Air Zoom Pegasus 40
Description: Բարձր արդյունավետ վազքի կոշիկներ՝ արձագանքող մեղմացնող միջադիրներով, շնչող ցանցավոր վերևով և բարեկարգված տեղադրմամբ՝ պրոֆեսիոնալ երկար վազքով և մարզական պատրաստությամբ զբաղվողների համար:


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Query: ռոբոտ փոշեկուլ
Most similar item in the dataset:
Name: iRobot Roomba j7+
Description: Առաջադեմ ռոբոտ փոշեկուլ՝ խելացի նավիգացիայով, խոչընդոտներից խուսափելու ունակությամբ, ինքնաթափ հիմքով և տնային խելացի համակարգերի ինտեգրացիայով՝ առանց ջանքի մաքրման համար:
